# Demo Notebook:
## Single Risk Survival Transformer For Causal Sequence Modelling 

Including time, tabular values

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

!pwd

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/modelling/SurvStreamGPT/notebooks/CompetingRisk


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
from pycox.evaluation import EvalSurv
from tqdm import tqdm
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

Using device: cuda.


# Demo Version of SurvStreamGPT

## Build configurations

In [5]:
# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../../confs", job_name="testing_notebook"):
    cfg = compose(config_name="config_SingleRisk11M", 
                  # overrides=[
                  #     ]
                 )

# Just load in pretrained model
cfg.experiment.train = False
cfg.experiment.test = False
cfg.experiment.log = False
print(OmegaConf.to_yaml(cfg))

# save_path = "Projects/CPRD/examples/modelling/SurvStreamGPT/notebooks/CompetingRisk/"

save_path = "/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/SR_11M_1337/"

is_decoder: true
data:
  batch_size: 64
  unk_freq_threshold: 0.0
  min_workers: 20
experiment:
  project_name: SurvStreamGPT_${head.SurvLayer}
  run_id: PreTrain_${head.SurvLayer}_129M_${experiment.seed}
  train: false
  test: false
  verbose: true
  seed: 1337
  log: false
  log_dir: /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/
  ckpt_dir: /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/
optim:
  num_epochs: 1
  learning_rate: 0.0001
  val_check_interval: 1000
  early_stop_patience: 5
  log_every_n_steps: 20
  limit_val_batches: 0.05
  limit_test_batches: 0.05
transformer:
  block_type: Neo
  block_size: 128
  n_layer: 10
  n_head: 8
  n_embd: 1024
  layer_norm_bias: false
  attention_type: global
  bias: true
  dropout: 0.2
  attention_dropout: 0.2
  resid_dropout: 0.2
head:
  SurvLayer: cr
  surv_weight: 0.5
  tokens_for_univariate_regression: None
  value_weight: 0.5



In [6]:
 # TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28      

model, dm = run(cfg)     
print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")


INFO:root:Running cr experiment on 72 CPUs and 1 GPUs
INFO:root:Loading Polars dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/polars/


env: SLURM_NTASKS_PER_NODE=28


INFO:root:Using tokenizer tabular
INFO:root:Tokenzier created based on 3584.43M tokens
INFO:root:Creating split=train/ dataset
INFO:root:	 Loading split=train/ hash map for parquet
INFO:root:	 Hash map created for split=train/ with 22,912,046 samples
INFO:root:Creating split=test/ dataset
INFO:root:	 Loading split=test/ hash map for parquet
INFO:root:	 Hash map created for split=test/ with 1,207,449 samples
INFO:root:Creating split=val/ dataset
INFO:root:	 Loading split=val/ hash map for parquet
INFO:root:	 Hash map created for split=val/ with 1,226,576 samples
INFO:root:184 vocab elements
INFO:root:Using Temporal Positional Encoding. This module uses the patient's age at an event within their time series.
INFO:root:Using Competing-Risk DeSurv head.
INFO:root:Internally scaling time in survival head by 1825 days
INFO:root:In generation forwarding DeSurv on the grid between [0.0, 1825.0] with 1826 intervals of delta=1.0
GPU available: True (cuda), used: True
TPU available: False, using:

Loaded model with 129.024283 M parameters


In [7]:
dm.train_set.view_sample(100, report_time=True)

# for batch in dm.train_dataloader():
#     break
# print(batch)

Time to retrieve sample index 100 was 0.13484811782836914 seconds

SEX                 | M
IMD                 | 4.0
ETHNICITY           | MIXED
birth_year          | 1991.0

Token                                                                      | Age               | Standardised value
Haemoglobin_estimation_9                                                   | 6405              | nan               
Platelet_count_12                                                          | 6405              | -0.17             
Serum_creatinine_31                                                        | 6405              | 0.17              
Serum_potassium_26                                                         | 6405              | 0.13              
Serum_sodium_24                                                            | 6405              | -0.01             
Serum_urea_level_29                                                        | 6405              | -0.17             
Total_white_c

In [8]:
import polars as pl
pl.Config.set_tbl_rows(200)
pl.Config.set_fmt_str_lengths(100)
display(dm.tokenizer._event_counts)

EVENT,COUNT,FREQUENCY
str,u32,f64
"""UNK""",0,0.0
"""ADDISONS_DISEASE""",6691,0.000002
"""CYSTICFIBROSIS""",7053,0.000002
"""SYSTEMIC_SCLEROSIS""",8772,0.000002
"""SICKLE_CELL_DISEASE_V2""",11159,0.000003
"""ADDISON_DISEASE""",11794,0.000003
"""DOWNSSYNDROME""",17006,0.000005
"""HAEMOCHROMATOSIS_V2""",18631,0.000005
"""PLASMACELL_NEOPLASM_V2""",20301,0.000006


### Real data

In [9]:
display(dm.meta_information["measurement_tables"])

,event,count,count_obs,digest,min,max,mean,approx_lqr,approx_uqr
0,25_Hydroxyvitamin_D2_level_92,782791,693470,"({'m': 0.0, 'c': 9.0}, {'m': 0.1, 'c': 112.0},...",0.000,6.860000e+02,3.908721,-4.646246,10.806670
1,25_Hydroxyvitamin_D3_level_90,809104,781118,"({'m': 0.1, 'c': 3.0}, {'m': 1.0, 'c': 314.0},...",0.000,9.518000e+02,47.148892,-36.374906,121.281425
2,AST___aspartate_transam_SGOT__46,1738489,1680613,"({'m': 0.0, 'c': 3901.0}, {'m': 0.770571428571...",0.000,1.533000e+04,26.619633,3.417134,41.771075
3,AST_serum_level_47,10837982,10485351,"({'m': 0.0, 'c': 53.0}, {'m': 1.8, 'c': 1.0}, ...",-5.000,2.070000e+04,27.251680,4.558863,41.966985
4,Albumin___creatinine_ratio_37,180911,78420,"({'m': -1.0, 'c': 1.0}, {'m': 0.0, 'c': 4213.0...",-1.000,1.282100e+04,10.672548,-4.327568,8.831512
...,...,...,...,...,...,...,...,...,...
103,Total_cholesterol_HDL_ratio_95,15760772,15489099,"({'m': 0.0, 'c': 21.0}, {'m': 0.001, 'c': 2.0}...",-3.100,3.720369e+09,964.704260,0.503984,6.883091
104,Total_white_cell_count_18,94827537,94179010,"({'m': -0.3, 'c': 1.0}, {'m': 0.0, 'c': 3.0}, ...",-14.700,3.720369e+09,250.189546,1.595425,12.090323
105,Urine_albumin_creatinine_ratio_35,15107807,10249206,"({'m': 0.0, 'c': 213.0}, {'m': 0.01, 'c': 3.0}...",-14.000,1.000000e+10,996.069857,-3.867288,8.042548
106,Urine_microalbumin_creatinine_ratio_36,201318,94009,"({'m': 0.0, 'c': 5691.0}, {'m': 0.001, 'c': 53...",-5.803,3.176800e+04,10.775029,-4.142800,8.260179


## Generation

### Sampling from the model

In [10]:
# Default context start
baseline_covariates = {"sex": "F", "deprivation": 1.0, "ethnicity": "WHITE", "year_of_birth": 1997-40}
prompt = ["O_E___height_1", "O_E___weight_2"]
values = [163, 80]
ages_in_years = [18.2, 48.2]

# define encoding functions (TODO: add this wrap to datamodule
encode_prompt = lambda prompt_list: torch.from_numpy(np.array(dm.encode(prompt_list)).reshape((1,-1))).to(device)
encode_value = lambda prompt_list, value_list: torch.tensor(np.array([dm.standardise(_cat, _val) for _cat, _val in zip(prompt_list, value_list) ]).reshape((1,-1)), dtype=torch.float32).to(device)
encode_age = lambda age_list: torch.tensor([365 * _age for _age in age_list], dtype=torch.int64).reshape((1,-1)).to(device)

# Convert for model
covariates = dm.train_set._encode_covariates(**baseline_covariates).reshape(1,-1).to(device)
tokens = encode_prompt(prompt)
values_scaled = encode_value(prompt, values)
ages_in_days = encode_age(ages_in_years)

print(values_scaled)

tensor([[-0.0444,  0.0650]], device='cuda:0')


In [11]:
# generate: sample the next 10 tokens
new_tokens, new_ages, new_values = model.generate(tokens, ages_in_days, values_scaled, covariates, max_new_tokens=40)

# report:
print(f"Baseline covariates: \n{baseline_covariates}\n" + "="*90)
print(f"PROMPT:")
for _idx, (_cat, _age, _value) in enumerate(zip(dm.decode(new_tokens[0].tolist()).split(" "), 
                                                new_ages[0, :], 
                                                new_values[0, :]
                                               )
                                           ):
    # _value = dm.unstandardise(_cat, _value)
    print(f"{_cat}".ljust(50) + f"{_value:.02f}".ljust(15) + f"at age {_age/365:.0f} ({_age:.1f} days)")    # with value {_value}
    if _idx == tokens.shape[-1] - 1:
        print("="*90)
        print(f"GENERATION")

Baseline covariates: 
{'sex': 'F', 'deprivation': 1.0, 'ethnicity': 'WHITE', 'year_of_birth': 1957}
PROMPT:
O_E___height_1                                    -0.04          at age 18 (6643.0 days)
O_E___weight_2                                    0.06           at age 48 (17593.0 days)
GENERATION
O_E___weight_2                                    0.08           at age 48 (17594.0 days)
O_E___weight_2                                    0.18           at age 48 (17596.0 days)
Systolic_blood_pressure_4                         -0.03          at age 48 (17596.0 days)
Basophil_count_22                                 0.04           at age 51 (18498.0 days)
Corrected_serum_calcium_level_42                  0.05           at age 51 (18500.0 days)
Eosinophil_count_21                               -0.14          at age 51 (18501.0 days)
Haematocrit_15                                    -0.08          at age 51 (18504.0 days)
Haemoglobin_A1c_level___IFCC_standardised_6       -0.24          at age 

# Prompt testing

## Diagnoses: How related conditions are impacted by each other


In [12]:
exp_prompts = [["DEPRESSION"], ["TYPE1DM"], ["TYPE2DIABETES"], ["Never_smoked_tobacco_85"], ["Ex_smoker_84"]]
exp_ages = [[20] for _ in range(len(exp_prompts))]
exp_values = [[np.nan] for _ in range(len(exp_prompts))]

with torch.no_grad(): 
    model.eval()

    _exp_survs = []
    for p_idx, (_exp_prompt, _exp_age, _exp_value) in enumerate(zip(exp_prompts, 
                                                                    exp_ages, 
                                                                    exp_values)):

        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)
        
        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=covariates,
                                       is_generation=True)
        _exp_survs.append(surv)

    for si, _ in enumerate(surv):
        plt.close()
        event_name = dm.decode([si + 1])
        for p_idx in range(len(exp_prompts)):
            plt.plot(model.surv_layer.t_eval / 365, _exp_survs[p_idx][si][0, :], label=f"{'->'.join(exp_prompts[p_idx]).lower()}")
        plt.xlabel("Time (years)")
        plt.ylabel(f"$P(T>t)$ ({event_name})")
        plt.legend()
        plt.savefig(save_path + f"diabetes/{event_name}.png")


## Values: How increasing BMI affects diagnosis risk

In [13]:
events_of_interest = ["Body_mass_index_3", "Diastolic_blood_pressure_5", 
                      "TYPE1DM", "TYPE2DIABETES",
                      "HYPERTENSION", "OSTEOARTHRITIS",
                      "CKDSTAGE3TO5",
                      "HF_V3", "ISCHAEMICSTROKE_V2",
                      "DEATH"
                     ]

_exp_prompt = ["Body_mass_index_3"]
_exp_age = [40]
_exp_values = [[18.], [21.], [24.], [30.], [40.]]

with torch.no_grad(): 
    model.eval()

    _exp_survs = []
    for p_idx, _exp_value in enumerate(_exp_values):

        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)

        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=covariates,
                                       is_generation=True)
        _exp_survs.append(surv)

    for si, _ in enumerate(surv):
        plt.close()
        event_name = dm.decode([si + 1])
        if event_name in events_of_interest:
            for p_idx in range(len(_exp_values)):
                plt.plot(model.surv_layer.t_eval / 365, _exp_survs[p_idx][si][0, :], label=f"{_exp_values[p_idx][0]:.2f}")
            plt.xlabel("t (years)")
            plt.ylabel(f"$P(T>t)$ ({event_name})")
            plt.legend()
            plt.savefig(save_path + f"bmi/{event_name}.png")


## Values: How increasing DBP affects diagnosis risk

In [14]:
events_of_interest = ["Body_mass_index_3", "Diastolic_blood_pressure_5", 
                      "TYPE1DM", "TYPE2DIABETES",
                      "HYPERTENSION", "OSTEOARTHRITIS",
                      "CKDSTAGE3TO5",
                      "HF_V3", "ISCHAEMICSTROKE_V2",
                      "DEATH"
                     ]


_exp_prompt = ["Diastolic_blood_pressure_5"]
_exp_age = [40]
_exp_values = [[60.], [70.], [80.], [90.], [100.], [110.]]

with torch.no_grad(): 
    model.eval()

    _exp_survs = []
    for p_idx, _exp_value in enumerate(_exp_values):

        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)

        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=covariates,
                                       is_generation=True)
        _exp_survs.append(surv)

    for si, _ in enumerate(surv):
        plt.close()
        event_name = dm.decode([si + 1])
        if event_name in events_of_interest:
            for p_idx in range(len(_exp_values)):
                plt.plot(model.surv_layer.t_eval / 365, _exp_survs[p_idx][si][0, :], label=f"{_exp_values[p_idx][0]:.2f}")
            plt.xlabel("t (years)")
            plt.ylabel("P(T>t) ()")
            plt.legend()
            plt.savefig(save_path + f"diastolic_blood_pressure/{event_name}.png")


## Values: How varying diagnosis affects value of DBP

In [15]:
measurements_of_interest = "Diastolic_blood_pressure_5"


_exp_prompts = [["DEPRESSION"], ["TYPE2DIABETES"], ["HF_V3"], ["HYPERTENSION"]]
_exp_age = [20]
_exp_value = [np.nan]


with torch.no_grad(): 
    model.eval()

    for p_idx, _exp_prompt in enumerate(_exp_prompts):

        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)
        
        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=covariates,
                                       is_generation=True)

        dist = val_dist[model.value_layer.token_key(dm.tokenizer._stoi[measurements_of_interest])]
        print(f"{'->'.join(_exp_prompt)}".ljust(30) + "leads to".ljust(20) + f"standardised {measurements_of_interest} ~ N({dist.loc.item():.1f}, {dist.scale.item():.1f})")

# DEPRESSION                    leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
# TYPE2DIABETES                 leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
# HF_V3                         leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
# HYPERTENSION                  leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.0, 0.2)

DEPRESSION                    leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
TYPE2DIABETES                 leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.0, 0.2)
HF_V3                         leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
HYPERTENSION                  leads to            standardised Diastolic_blood_pressure_5 ~ N(0.1, 0.2)


## Values: How increasing bmi affects value of diastolic_blood_pressure

In [16]:
measurements_of_interest = "Diastolic_blood_pressure_5"


_exp_prompt = ["Body_mass_index_3"]
_exp_values = [[18.], [21.], [24.], [30.], [40.]]
_exp_value = [np.nan]


with torch.no_grad(): 
    model.eval()

    for p_idx, _exp_value in enumerate(_exp_values):

        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)
        
        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=covariates,
                                       is_generation=True)

        dist = val_dist[model.value_layer.token_key(dm.tokenizer._stoi[measurements_of_interest])]
        print(f"{'->'.join(_exp_prompt)} of {_exp_value[0]}".ljust(30) + "leads to".ljust(20) + f"standardised {measurements_of_interest} ~ N({dist.loc.item():.1f}, {dist.scale.item():.1f})")

# Body_mass_index_3 of 18.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.2, 0.2)
# Body_mass_index_3 of 21.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.2, 0.2)
# Body_mass_index_3 of 24.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
# Body_mass_index_3 of 30.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
# Body_mass_index_3 of 40.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(0.0, 0.2)

Body_mass_index_3 of 18.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
Body_mass_index_3 of 21.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
Body_mass_index_3 of 24.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.1, 0.2)
Body_mass_index_3 of 30.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(-0.0, 0.2)
Body_mass_index_3 of 40.0     leads to            standardised Diastolic_blood_pressure_5 ~ N(0.0, 0.2)


## Baseline, impact of gender

In [17]:
events_of_interest = ["Body_mass_index_3", "Diastolic_blood_pressure_5", 
                      "TYPE1DM", "TYPE2DIABETES",
                      "HYPERTENSION", "OSTEOARTHRITIS",
                      "CKDSTAGE3TO5",
                      "HF_V3", "ISCHAEMICSTROKE_V2",
                      "POLYCYSTIC_OVARIAN_SYNDROME_PCOS_V2",
                      "DEATH"
                     ]

_genders = ["M", "F", "I"]
_exp_prompt = ["Diastolic_blood_pressure_5"]
_exp_age = [20]
_exp_value = [90.]

with torch.no_grad(): 
    model.eval()

    _exp_survs = []
    for p_idx, _gender in enumerate(_genders):

        _baseline_covariate = {"sex": _gender, "deprivation": 4.0, "ethnicity": "WHITE", "year_of_birth": 1997}
        _covariates = dm.train_set._encode_covariates(**_baseline_covariate).reshape(1,-1).to(device)
        _tokens = encode_prompt(_exp_prompt)
        _values_scaled = encode_value(_exp_prompt, _exp_value)
        _ages_in_days = encode_age(_exp_age)

        (surv, val_dist), _, _ = model(_tokens,
                                       values=_values_scaled,
                                       ages=_ages_in_days,
                                       covariates=_covariates,
                                       is_generation=True)
        _exp_survs.append(surv)

    for si, _ in enumerate(surv):
        plt.close()
        event_name = dm.decode([si + 1])
        if event_name in events_of_interest:
            for p_idx in range(len(_genders)):
                plt.plot(model.surv_layer.t_eval / 365, _exp_survs[p_idx][si][0, :], label=f"{_genders[p_idx]}")
            plt.xlabel("t (years)")
            plt.ylabel("P(T>t) ()")
            plt.legend()
            plt.savefig(save_path + f"gender/{event_name}.png")


# Appendix: model architectures

In [18]:
display(model)

SurvStreamGPTForCausalModelling(
  (transformer): TTETransformer(
    (wpe): TemporalPositionalEncoding()
    (wte): DataEmbeddingLayer(
      (static_proj): Linear(in_features=16, out_features=1024, bias=True)
      (dynamic_embedding_layer): SplitDynamicEmbeddingLayer(
        (cat_event_embed_layer): Embedding(184, 1024, padding_idx=0)
        (cat_event_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (num_value_embed_layer): EmbeddingBag(184, 1024, mode='sum', padding_idx=0)
        (num_value_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (drop): Dropout(p=0.2, inplace=False)
    (blocks): ModuleList(
      (0-9): 10 x Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadedSelfAttention(
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
    

In [19]:
!jupyter nbconvert --to html --no-input generation.ipynb

[NbConvertApp] Converting notebook generation.ipynb to html
[NbConvertApp] Writing 627138 bytes to generation.html
